In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

In [2]:
import numpy as np
from tqdm.notebook import trange
from tqdm.notebook import tqdm
from call_ledger import timer as T

np.random.seed(13)

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [3]:
C = 10 # number of campaigns
U = 5000 # number of customers.
H = 4 # number of channels.
D = 7 # number of planning days.
I = 3 # number of quota categories.
P = 10 # number of priority categories.

#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
e_cu = np.random.choice(2,(C, U))
#e_cu = np.ones((C, U), dtype='int8')
e_cu_X = np.stack([np.stack([e_cu for _ in range(H)], axis=2) for _ in range(D)], axis=3)
e_cu

array([[0, 0, 0, ..., 1, 0, 1],
       [0, 1, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 1, 1],
       ...,
       [1, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 1, 0]])

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [5]:
q_ic = np.random.choice(2, (I,C))
#q_ic = np.zeros((I,C), dtype='int8')
q_ic

array([[1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [1, 1, 1, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 1, 1]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [6]:
r_p = np.random.choice(100, P)
#r_p = np.ones(P, dtype='int8')
rp_c = np.array([r_p[r] for r in np.random.choice(P, C)])
rp_c

array([43, 47, 52, 47, 96, 43, 43, 43, 96, 47])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [7]:
b = 7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [8]:
k = 3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [9]:
l_c = np.random.choice([2,3,4],C)
l_c

array([2, 3, 2, 2, 3, 4, 2, 4, 2, 4])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [10]:
m_i = np.random.choice([4,3,5],I)
m_i_X = np.stack([m_i for _ in range(U)], axis=1)
n_i = np.random.choice([1,3,2],I)
n_i_X = np.stack([n_i for _ in range(U)], axis=1)
#m_i = np.ones((I), dtype='int8')*10
#n_i = np.ones((I), dtype='int8')*10

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [11]:
t_hd = np.random.choice([U*.7, U*.6, U*.5], (H, D))

# Model

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [12]:
X_cuhd = np.zeros((C,U,H,D), dtype='int')

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall h \in H,\forall d \in D
$$

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

In [13]:
%%timeit
all([(X_cuhd[:,:,h,d] <= e_cu).all() for h in range(H) for d in range(D)])

9.62 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit
np.all(X_cuhd <= e_cu_X)

1.95 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit
(X_cuhd <= e_cu_X).all()

1.79 ms ± 96.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [16]:
%%timeit
np.all(X_cuhd.sum(axis=(2)).sum(axis=(2)).T<=l_c)

8.51 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit
np.all(X_cuhd.sum(axis=(2,3)).T<=l_c)

976 µs ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [18]:
@T.timeit
def eligibility (X):
    return (X <= e_cu_X).all()

#@T.timeit
#def one_channel (X):
#    return np.all(X.sum(axis=2)<=1)

@T.timeit
def weekly_limitation (X):
    return (X.sum(axis=(0,2,3))<=b).all()

@T.timeit
def daily_limitation (X):
#    return (X.sum(axis=(0,2))<=k).all()
    return (X.sum(axis=(0)).sum(axis=(1))<=k).all()

@T.timeit
def campaign_limitation (X):
    return np.all(X.sum(axis=(2,3)).T<=l_c)

@T.timeit
def weekly_quota (X):
    for i in range(I):
        quota_i = np.all(X_cuhd.sum(axis=(2,3)).T * q_ic[i, ].T  <= m_i[i])
        if not quota_i:
            return False
    return True

@T.timeit
def channel_capacity (X):
    return np.all(X.sum(axis=(0,1))<=t_hd)

@T.timeit
def daily_quota (X):
    for i in range(I):
        quota_i = np.all((q_ic[0,].T * X.sum(axis=2).T).sum(2) <= n_i[0])
        if not quota_i:
            return False
    return True

%%timeit
np.all((q_icuhd*X_cuhd).sum(axis=(1,3)) <= n_icuhd)

%%timeit
all([np.all((q_ic[i,].T * X_cuhd.sum(axis=2).T).sum(2) <= n_i[i]) for i in range(I)])

%%timeit
np.all((q_ic[0,].T * X_cuhd.sum(axis=2).T).sum(2) <= n_i[0])
np.all((q_ic[1,].T * X_cuhd.sum(axis=2).T).sum(2) <= n_i[1])
np.all((q_ic[2,].T * X_cuhd.sum(axis=2).T).sum(2) <= n_i[2])

def a():
    print('a')
    return False
def b():
    print('b')
    return False

np.all([a(),b()])
all([a(),b()])


##### - check criterion

In [19]:
from multiprocessing.pool import ThreadPool
pool = ThreadPool(8)

@T.timeit
def check(X):
#    ff = [pool.apply_async(eligibility, args=(X,)),
#    pool.apply_async(one_channel, args=(X,)),
#    pool.apply_async(weekly_limitation, args=(X,)),
#    pool.apply_async(daily_limitation, args=(X,)),
#    pool.apply_async(campaign_limitation, args=(X,)),
#    pool.apply_async(weekly_quota, args=(X,)),
#    pool.apply_async(daily_quota, args=(X,)),
#    pool.apply_async(channel_capacity, args=(X,))]
#    for r in ff:
#        if not r.get():
#            return False
#    return True   
    if not eligibility(X):
        return False
    if not weekly_limitation(X):
        return False
    if not daily_limitation(X):
        return False
    if not campaign_limitation(X):
        return False
    if not weekly_quota(X):
        return False
    if not daily_quota(X):
        return False
    if not channel_capacity(X):
        return False
#    if not one_channel(X):
#        return False
    return True

In [20]:
%%time
X_cuhd = np.zeros((C,U,H,D), dtype='int')
for c in tqdm(np.argsort(-rp_c), desc="Campaigns Loop"):
    for d in trange(D, desc=f"Days Loop for campaign-{c}"):
        for h in range(H):#trange(H, desc=f"Channels Loop at Day-{d}, Campapaign-{c}"):
            for u in range(U):#trange(U, desc=f"Users Loop On Campaign-{c}"):
                X_cuhd[c,u,h,d]=1
                if not check(X_cuhd):
                    X_cuhd[c,u,h,d]=0

Campaigns Loop:   0%|          | 0/10 [00:00<?, ?it/s]

Days Loop for campaign-4:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-8:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-2:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-1:   0%|          | 0/7 [00:00<?, ?it/s]

...

Days Loop for campaign-0:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-5:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-6:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-7:   0%|          | 0/7 [00:00<?, ?it/s]

Wall time: 2h 21min 33s


In [21]:
np.matmul(rp_c, X_cuhd.sum(axis=(1,2,3)))

2229756

In [22]:
for k,v in T.DURATION_STATS.items():
    print(k,v['duration']/v['counter'],v['counter'])

eligibility 0.002322330914714741 1400000
check 0.006060030144642743 1400000
weekly_limitation 0.001316560486768754 700168
daily_limitation 0.0019245086816460422 335259
campaign_limitation 0.0012266008674101166 306199
weekly_quota 0.00408274733289169 96284
daily_quota 0.028404331060195534 96284
channel_capacity 0.0017663645317397977 82251


- Sort Campaigns by rp_c
- Map every criteria to function

##kucuk sayilar ile algoritmayi test edelim, ve bb solution burada validate edelim.